# Answer for question 1, week 3


In [5]:
!pip install BeautifulSoup4
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis 
import codecs
from bs4 import BeautifulSoup

!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print("Done")


--2019-08-11 00:55:02--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79284 (77K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M.8’

List_of_postal_code 100%[===================>]  77.43K  --.-KB/s    in 0.06s   

2019-08-11 00:55:02 (1.37 MB/s) - ‘List_of_postal_codes_of_Canada:_M.8’ saved [79284/79284]

Done


In [9]:
f1 = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html = f1.read()
soup = BeautifulSoup(html, 'html.parser')

df = (pd.read_html(str(soup.table))[0])

#drop the first row
df.drop(axis=0, index=0, inplace=True)

#then assign column names
column_names = ['Postcode', 'Borough', 'Neighborhood'] 
df.columns = column_names

#remove unassigned Boroughs
df = df[df['Borough'] != "Not assigned"].reset_index(drop=True) 
df

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Now, for an unassigned neighborhood. Give it the same name as its borough. A loop for those kind of data in general.

In [11]:
na = df[df['Neighborhood']=="Not assigned"]

for index in range(len(na)):
    row = na.iloc[index]
    row['Neighborhood']=row['Borough']
    df.iloc[na.index[0]]['Neighborhood'] = row['Borough']

df.head(8)   

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue


Now, there are many neighborhoods with the same borough. so we will use the postal code to group it and arrange them with same borough.

In [15]:
aggregations = {
    'Neighborhood':lambda x:', '.join(x)}

grp = df.groupby('Postcode').agg(aggregations).reset_index()
a = df.drop('Neighborhood',axis=1)
b = pd.merge(a, grp, on='Postcode', how='left')
data = b.drop_duplicates()
data.to_csv("Toronto_Q1_Answer.csv", index=False)
data

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


In [16]:
data.shape

(103, 3)